# Stable Baselines3 (SB3) - 可靠的强化学习算法实现教程

欢迎来到 Stable Baselines3 (SB3) 教程！Stable Baselines3 是一个基于 PyTorch 的开源库，提供了可靠、易于使用的常见强化学习 (RL) 算法实现。

**为什么使用 Stable Baselines3？**

1.  **可靠的实现**: 提供了经过良好测试和基准测试的标准 RL 算法（如 PPO, A2C, DQN, SAC, TD3）。
2.  **易于使用**: API 设计简洁，可以快速地在 Gymnasium 环境上训练和评估 RL Agent。
3.  **文档和社区**: 拥有良好的文档和活跃的社区支持。
4.  **基于 PyTorch**: 底层使用 PyTorch 构建，可以方便地与 PyTorch 生态系统集成，并支持自定义网络结构。
5.  **标准化**: 遵循与 Gymnasium 兼容的接口。

SB3 是快速应用、测试和比较标准 RL 算法的绝佳工具，特别适合初学者和需要可靠基线的开发者。

**本教程将涵盖 Stable Baselines3 的核心用法：**

1.  安装 Stable Baselines3
2.  基本工作流程：环境 -> 模型 -> 训练 -> 评估 -> 预测
3.  加载和使用预定义的 RL 算法
4.  训练模型 (`model.learn()`)
5.  模型评估 (`evaluate_policy`)
6.  模型预测 (`model.predict()`)
7.  模型保存与加载
8.  (简介) 回调函数 (Callbacks)

## 1. 安装 Stable Baselines3

你需要先安装 SB3。推荐安装带有 `extra` 的版本，它包含了 PyTorch 等核心依赖。

```bash
pip install stable-baselines3[extra]
# 或者只安装核心和 PyTorch (如果已安装 PyTorch，可能只需 pip install stable-baselines3)
```
同时，确保你已经安装了 Gymnasium 和你想使用的环境，例如：
```bash
pip install gymnasium gymnasium[classic_control]
```

In [ ]:
import gymnasium as gym
import numpy as np
import os

# 从 stable_baselines3 导入常用算法
from stable_baselines3 import PPO, A2C, DQN, SAC, TD3
from stable_baselines3.common.env_util import make_vec_env # 用于创建向量化环境 (并行)
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor # 用于包装环境以记录统计信息
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_checker import check_env # 检查自定义环境是否符合规范

print("Stable Baselines3 and Gymnasium imported.")

## 2. 基本工作流程

使用 SB3 的典型流程如下：
1.  **创建环境**: 使用 `gymnasium.make()` 创建一个（或多个，用于向量化）环境实例。
2.  **实例化模型**: 从 SB3 选择一个算法（如 `PPO`, `A2C`, `DQN`），并用环境和超参数实例化它。
3.  **训练模型**: 调用 `model.learn(total_timesteps=...)` 来训练 Agent。
4.  **(可选) 保存模型**: 使用 `model.save(path)` 保存训练好的 Agent。
5.  **(可选) 加载模型**: 使用 `ModelClass.load(path)` 加载之前保存的 Agent。
6.  **评估模型**: 使用 `evaluate_policy(model, env, n_eval_episodes=...)` 评估 Agent 性能。
7.  **使用模型进行预测**: 调用 `model.predict(observation, deterministic=True)` 获取给定观测下的最佳动作。

## 3. 加载和使用预定义的 RL 算法

SB3 提供了多种预定义的算法。选择哪种算法取决于你的环境（动作空间是离散还是连续）和具体任务。

*   **离散动作空间常用**: DQN, A2C, PPO
*   **连续动作空间常用**: A2C, PPO, SAC, TD3

In [ ]:
print("--- Creating Environment and Model Instance ---")

# 1. 创建环境 (例如 CartPole，离散动作空间)
env_id = 'CartPole-v1'
try:
    # 使用 Monitor 包装器来记录回合奖励和长度
    # 创建一个函数来生成环境，这对于向量化是必需的
    def make_cartpole_env():
        env = gym.make(env_id)
        env = Monitor(env) # Wrap with Monitor
        return env
        
    # 使用 DummyVecEnv 创建单个环境的向量化版本 (API一致性)
    # 对于并行环境，可以使用 SubprocVecEnv 或 make_vec_env
    vec_env = DummyVecEnv([make_cartpole_env])
    print(f"Environment '{env_id}' created and wrapped.")
    
    # 检查环境是否符合 SB3/Gymnasium 规范 (可选)
    # check_env(env) # Check the original env instance

    # 2. 实例化模型 (例如 PPO)
    # policy='MlpPolicy' 使用默认的多层感知机作为策略网络
    # verbose=1 打印训练过程中的信息
    model = PPO('MlpPolicy', vec_env, verbose=0, seed=42)
    print(f"PPO model instantiated with 'MlpPolicy'. Policy architecture:")
    print(model.policy)

except gym.error.NameNotFound as e:
    print(f"Error creating environment: {e}. Please install gymnasium[classic_control]")
    model = None
    vec_env = None
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    model = None
    vec_env = None

## 4. 训练模型 (`model.learn()`)

调用 `learn()` 方法开始训练。你需要指定总的训练步数 (`total_timesteps`)。

In [ ]:
print("--- Training the Model ---")

if model and vec_env:
    total_training_steps = 10000 # 减少步数以便快速演示
    print(f"Starting training for {total_training_steps} timesteps...")
    
    # 实际训练过程
    # log_interval 控制打印日志的频率 (每隔多少个回合)
    # progress_bar=True 显示一个进度条 (需要 tqdm 安装)
    try:
        model.learn(total_timesteps=total_training_steps, progress_bar=True)
        print("Training finished.")
        training_successful = True
    except Exception as e:
        print(f"An error occurred during training: {e}")
        training_successful = False
else:
    print("Model or environment not available, skipping training.")
    training_successful = False

## 5. 模型评估 (`evaluate_policy`)

使用独立的测试环境来评估训练好的 Agent 的性能（平均奖励和标准差）。

In [ ]:
print("--- Evaluating the Trained Model ---")

if training_successful and vec_env: # Check if training likely completed
    # 创建一个新的评估环境 (最好与训练环境分开)
    # 不需要向量化，但需要 Monitor 来记录奖励
    eval_env = None
    try:
        eval_env = gym.make(env_id) 
        eval_env = Monitor(eval_env)
        print("Evaluation environment created.")

        # n_eval_episodes: 评估的回合数
        # deterministic=True: 使用确定性策略进行评估 (通常性能更好)
        mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

        print(f"\nEvaluation Results (10 episodes, deterministic):")
        print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")
        
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")
    finally:
        if eval_env:
             eval_env.close()
             print("Evaluation environment closed.")
else:
    print("Skipping evaluation as model training did not complete successfully or env is missing.")

## 6. 模型预测 (`model.predict()`)

使用训练好的模型来根据当前观测选择动作。

In [ ]:
print("--- Using the Model for Prediction ---")

if training_successful and vec_env: # Use the vec_env used for training or create a new one
    obs, info = vec_env.reset() # VecEnv reset returns observation for each env
    print(f"Initial observation (from VecEnv): {obs}")
    
    # predict() 返回选择的动作和 (可选的) 内部状态 (对于 RNN/LSTM)
    # deterministic=True: 选择概率最高的动作
    # deterministic=False: 根据策略概率分布进行采样
    action, _states = model.predict(obs, deterministic=True)
    print(f"Predicted deterministic action for initial obs: {action}")
    
    action_stochastic, _ = model.predict(obs, deterministic=False)
    print(f"Predicted stochastic action for initial obs: {action_stochastic}")
    
    # 模拟一个回合的可视化 (如果环境支持渲染)
    print("\nSimulating one episode with the trained agent...")
    render_env = None
    try:
        # Use render_mode='human' for a pop-up window (might not work in all notebooks)
        # Use render_mode='rgb_array' to get frames for display/saving
        render_env = gym.make(env_id, render_mode='rgb_array') # Or 'human'
        obs, info = render_env.reset()
        frames = []
        total_reward_render = 0
        for _ in range(200): # Limit steps for demo
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = render_env.step(action)
            total_reward_render += reward
            frame = render_env.render()
            if frame is not None: frames.append(frame)
            if terminated or truncated:
                break
        print(f"Simulated episode finished. Total reward: {total_reward_render}")
        if frames:
             print(f"Collected {len(frames)} frames for potential visualization.")
             # Display the last frame as an example
             plt.figure(figsize=(4,3))
             plt.imshow(frames[-1])
             plt.title(f"Last Frame (Total Reward: {total_reward_render})")
             plt.axis('off')
             plt.show()
    except ImportError as e:
         print(f"Rendering failed, likely missing dependencies (e.g., for display): {e}")
    except Exception as e:
        print(f"An error occurred during rendering simulation: {e}")
    finally:
        if render_env:
             render_env.close()
             print("Render environment closed.")

else:
    print("Skipping prediction example.")

## 7. 模型保存与加载

可以将训练好的模型（包括策略网络、值函数网络、优化器状态等）保存到文件中，以便稍后加载和使用。

In [ ]:
print("--- Saving and Loading the Model ---")

model_save_path = "ppo_cartpole_model.zip"

if training_successful and model:
    # 保存模型
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")
    
    # 删除当前模型实例 (模拟在不同脚本或时间加载)
    del model 
    print("Current model instance deleted.")
    
    # 加载模型
    # 需要指定算法类 (PPO)
    # env 参数是可选的，但如果提供，可以自动设置模型的 action/observation space
    loaded_model = PPO.load(model_save_path, env=vec_env) # Pass the same type of env used for training
    print(f"Model loaded from {model_save_path}")
    
    # 使用加载的模型进行评估 (验证加载是否成功)
    print("Evaluating the loaded model...")
    eval_env_load = None
    try:
        eval_env_load = Monitor(gym.make(env_id))
        mean_reward_load, std_reward_load = evaluate_policy(loaded_model, eval_env_load, n_eval_episodes=5)
        print(f"Loaded Model Mean reward: {mean_reward_load:.2f} +/- {std_reward_load:.2f}")
    except Exception as e:
         print(f"Error during loaded model evaluation: {e}")
    finally:
        if eval_env_load:
             eval_env_load.close()

    # 清理保存的文件
    if os.path.exists(model_save_path):
        os.remove(model_save_path)
        print(f"Cleaned up {model_save_path}")
        
else:
    print("Skipping saving/loading example.")

## 8. (简介) 回调函数 (Callbacks)

SB3 允许使用回调函数 (`Callbacks`) 来自定义训练过程，例如：
*   在训练期间定期评估模型并保存最佳模型。
*   记录自定义指标。
*   提前停止训练。

常用的回调有 `EvalCallback`, `StopTrainingOnRewardThreshold` 等。

**示例 (概念):**
```python
# from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

# # 回调：在训练期间评估模型，并保存最佳模型
# eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/best_model',
#                            log_path='./logs/', eval_freq=500,
#                            deterministic=True, render=False)

# # 回调：当达到某个奖励阈值时停止训练
# reward_threshold_callback = StopTrainingOnRewardThreshold(reward_threshold=490, verbose=1)

# # 在 learn() 方法中传递回调列表
# model.learn(total_timesteps=20000, callback=[eval_callback, reward_threshold_callback])
```

## 总结

Stable Baselines3 是一个强大且易于使用的强化学习库，它极大地简化了标准 RL 算法的应用和评估。

**关键要点：**
*   提供了多种可靠的预实现 RL 算法。
*   遵循**环境 -> 模型 -> 训练 -> 评估/预测**的工作流程。
*   与 Gymnasium 环境无缝集成。
*   `learn()` 用于训练，`predict()` 用于获取动作，`evaluate_policy()` 用于评估。
*   支持模型保存和加载。
*   可以使用回调函数自定义训练过程。

对于想要快速开始应用强化学习算法或需要可靠基线的用户来说，SB3 是一个极好的起点。要进行更深入的研究或实现新算法，可能需要直接使用 PyTorch 等底层框架。